In [2]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , accuracy_score
import numpy as np

In [3]:
DATA_DIR = './train'

In [5]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

data = []
labels = []
for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

In [5]:
data_dict = pickle.load(open('./data.pickle', 'rb'))

In [21]:
# First filter the data
data_dict = pickle.load(open('./data.pickle', 'rb'))

# Find the most common length in your data
lengths = [len(d) for d in data_dict['data']]
from collections import Counter
common_length = Counter(lengths).most_common(1)[0][0]
print(f"Most common data length: {common_length}")

# Filter to keep only entries with that length
good_indices = [i for i, d in enumerate(data_dict['data']) if len(d) == common_length]
filtered_data = [data_dict['data'][i] for i in good_indices]
filtered_labels = [data_dict['labels'][i] for i in good_indices]

# Now convert to numpy arrays - just like you did before, but with the filtered data
data = np.asarray(filtered_data)
labels = np.asarray(filtered_labels)

print(f"Original data size: {len(data_dict['data'])}")
print(f"Filtered data size: {len(filtered_data)}")

Most common data length: 42
Original data size: 4742
Filtered data size: 4728


In [22]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [23]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [25]:
print(classification_report(y_test , y_predict))

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        48
           B       1.00      1.00      1.00        52
           C       0.96      1.00      0.98        27
           D       1.00      1.00      1.00        29
           E       1.00      1.00      1.00        49
           F       1.00      1.00      1.00        45
           G       1.00      1.00      1.00        48
           H       1.00      1.00      1.00        23
           I       1.00      1.00      1.00        36
           K       1.00      0.98      0.99        49
           L       1.00      1.00      1.00        36
           M       1.00      1.00      1.00        47
           N       1.00      1.00      1.00        47
           O       1.00      0.97      0.99        36
           P       1.00      1.00      1.00        44
           Q       1.00      1.00      1.00        35
           R       0.98      1.00      0.99        42
           S       1.00    

In [26]:
f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

In [28]:
score = accuracy_score(y_predict, y_test)
print('{}% of samples were classified correctly !'.format(score * 100))

99.78858350951374% of samples were classified correctly !


In [3]:
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Set static_image_mode=False for video processing
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

labels_dict = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E',
    5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K',
    10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P',
    15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U',
    20: 'V', 21: 'W', 22: 'X', 23: 'Y'
}

# Create reverse dictionary for lookup
reverse_labels_dict = {v: k for k, v in labels_dict.items()}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    
    # Check if frame was successfully captured
    if not ret or frame is None:
        print("Failed to capture frame")
        continue

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        # Reset data for each detection
        data_aux = []
        x_ = []
        y_ = []
        
        # Use only the first detected hand
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # First collect all coordinates for normalization
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            x_.append(x)
            y_.append(y)

        # Then normalize and store
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x - min(x_))
            data_aux.append(y - min(y_))

        # Make sure we have the expected data length
        if len(data_aux) == 42:  # 21 landmarks × 2 coordinates
            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10
            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model.predict([np.asarray(data_aux)])
            
            # Handle string predictions directly
            if isinstance(prediction[0], str):
                predicted_character = prediction[0]
            else:
                # If the model actually predicts integers
                predicted_character = labels_dict[prediction[0]]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    
    # Press 'q' to quit
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()